In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

## Simple python 2D heat equation solver using finite differences

In [8]:
print("2D heat equation solver")

plate_length = 50
max_iter_time = 350

alpha = 2
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

# Initialize solution: the grid of u(k, i, j)
u = np.empty((max_iter_time, plate_length, plate_length))

# Initial condition everywhere inside the grid
u_initial = 0

# Boundary conditions
u_top = 100.0
u_left = 0.0
u_bottom = 0.0
u_right = 100.0

# Set the initial condition
u.fill(u_initial)

# Set the boundary conditions
u[:, (plate_length-1):, :] = u_top
u[:, :, :1] = u_left
u[:, :1, 1:] = u_bottom
u[:, :, (plate_length-1):] = u_right

def calculate(u):
    for k in range(0, max_iter_time-1, 1):
        for i in range(1, plate_length-1, delta_x):
            for j in range(1, plate_length-1, delta_x):
                u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*u[k][i][j]) + u[k][i][j]

    return u

def plotheatmap(u_k, k):
    # Clear the current plot figure
    plt.clf()

    plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")
    plt.xlabel("x")
    plt.ylabel("y")

    # This is to plot u_k (u at time-step k)
    plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=0, vmax=100)
    plt.colorbar()

    return plt

# Do the calculation here
# time it 
import time
start_time = int(round(time.time() * 1000))

u = calculate(u)
end_time = int(round(time.time() * 1000))

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} ms")
# def animate(k):
#     plotheatmap(u[k], k)

# anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False)
# anim.save("heat_equation_solution.gif")

# print("Done!")

2D heat equation solver
Elapsed time: 2127.00 ms


## HPX and python implemetation 
Its the same python 2D heat equation solver using finite differences but with HPX doing some of the parts in parallel.

In [1]:
from pycxxwrap.pycxx import py11
from pycxxwrap.tools import set_args
from pycxxwrap.types import create_type

pybind11 headers path /home/matrix/.local/lib/python3.10/site-packages/pybind11/include
python headers path /usr/include/python3.10


In [11]:
a = set_args(flags = """-std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib""")
create_type("func","std::function<void()>")

In [12]:
@py11(headers=["<run_hpx.cpp>","<iostream>"],recompile=True,args=a)
def hpx_wrapper(f : func)->None:
    """
    const char *num = getenv("HPX_NUM_THREADS");
    int num_threads = num == 0 ? 4 : atoi(num);
    std::cout << "Using " << num_threads << " threads." << std::endl;
    hpx_global::submit_work(num_threads,f);
    """

self.flags =  -std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib
self.module_name =  False
self.lib_path =  False
input is ~/tmp
tilde
Directory already exists: /home/matrix/tmp
function name is : hpx_wrapper
/home/matrix/tmp/hpx_wrapper
type translation std::function<void()>
type translation void

                #include <pybind11/pybind11.h>
                #include <pybind11/stl.h>
                #include <run_hpx.cpp>
#include <iostream>

                

                namespace py = pybind11;

                extern "C" void hpx_wrapper(std::function<void()> f){
                
    const char *num = getenv("HPX_NUM_THREADS");
    int num_threads = num == 0 ? 4 : atoi(num);
    std::cout << "Using " << num_threads << " threads." << std::endl;
    hpx_global::submit_work(num_t

In [13]:
create_type("vvvec","std::vector<std::vector<std::vector<float>>>")

In [14]:
@py11(headers=["<iostream>",
               "<vector>",
               "<algorithm>",
               "<hpx/hpx_main.hpp>",
               "<hpx/include/parallel_for_each.hpp>"
               "<chrono>"],
    recompile=True,
    wrap=hpx_wrapper,
    args=a)
def optimized(u:vvvec,max_iter_time:int,plate_length:int, delta_x:int,gamma:float)->vvvec:
    """

    std::cout<<"max_iter_time"<<max_iter_time<<std::endl;
   

    
    auto start_time = std::chrono::high_resolution_clock::now();
    for (int k = 0; k < max_iter_time - 1; ++k) {
        hpx::for_each(hpx::execution::par, u[k].begin() + 1, u[k].end() - 1, [&](std::vector<float>& row) {
            int i = &row - &u[k][0];
            for (int j = 1; j < plate_length - 1; j += delta_x) {
                u[k + 1][i][j] = gamma * (u[k][i + 1][j] + u[k][i - 1][j] + u[k][i][j + 1] + u[k][i][j - 1] - 4 * u[k][i][j]) + u[k][i][j];
            }
        });
    }
    auto end_time = std::chrono::high_resolution_clock::now();
    auto elapsed_time = std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time);
    std::cout << "Elapsed time: " << elapsed_time.count() << " milliseconds" << std::endl;
    
    return u;
    """

self.flags =  -std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib
self.module_name =  False
self.lib_path =  False
input is ~/tmp
tilde
Directory already exists: /home/matrix/tmp
function name is : optimized
/home/matrix/tmp/optimized
type translation std::vector<std::vector<std::vector<float>>>
type translation int
type translation int
type translation int
type translation float
type translation std::vector<std::vector<std::vector<float>>>

                #include <pybind11/pybind11.h>
                #include <pybind11/stl.h>
                #include <iostream>
#include <vector>
#include <algorithm>
#include <hpx/hpx_main.hpp>
#include <hpx/include/parallel_for_each.hpp><chrono>

                
#include <dlfcn.h>
#include <fstream>
#include <sstream>
#include <stdlib.h>
#include <u

In [15]:
plate_length = 50
max_iter_time = 750

alpha = 8
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)
u = np.empty((max_iter_time, plate_length, plate_length))

# Initial condition everywhere inside the grid
u_initial = 0

# Boundary conditions
u_top = 100.0
u_left = 0.0
u_bottom = 0.0
u_right = 0.0

# Set the initial condition
u.fill(u_initial)

# Set the boundary conditions
u[:, (plate_length-1):, :] = u_top
u[:, :, :1] = u_left
u[:, :1, 1:] = u_bottom
u[:, :, (plate_length-1):] = u_right

def plotheatmap(u_k, k):
    # Clear the current plot figure
    plt.clf()

    plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")
    plt.xlabel("x")
    plt.ylabel("y")

    # This is to plot u_k (u at time-step k)
    plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=0, vmax=100)
    plt.colorbar()

    return plt

# CXX function call
u = optimized(u,max_iter_time,plate_length,delta_x,gamma)

Using 4 threads.
max_iter_time750
Elapsed time: 45 milliseconds
